<a href="https://colab.research.google.com/github/Anacatlisboa23/Application-of-Machine-Learning-Methodologies-in-the-Search-for-New-Peptides-with-Antimicrobial-Pote/blob/main/ESM_(Evolutionary_Scale_Modeling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESM (Evolutionary Scale Modeling)

In [ ]:
pip install fair-esm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import torch
import esm
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [ ]:
data=pd.read_csv("/content/feaurescertas_mic (1).csv")

In [ ]:
data

,Unnamed: 0,Sequence,Name,Target_Organism,MIC_Values,Gram,multiresistant,MIC_processado,length,A,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus,['0.2-0.8 µg/ml'],Gram-positivo,0,0.50,39.0,12.821,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2.63,0.0,0.0
1,1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae,['0.76 ug/ml'],Gram-negativo,0,0.76,37.0,8.108,...,0.0,0.0,0.0,0.0,2.78,0.0,0.0,0.00,0.0,0.0
2,2,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508,['0.38 ug/ml'],Gram-negativo,0,0.38,37.0,8.108,...,0.0,0.0,0.0,0.0,2.78,0.0,0.0,0.00,0.0,0.0
3,3,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus,['0.76 ug/ml'],Gram-positivo,0,0.76,37.0,8.108,...,0.0,0.0,0.0,0.0,2.78,0.0,0.0,0.00,0.0,0.0
4,4,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,['3.2 μg/ml'],Gram-negativo,0,3.20,62.0,16.129,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,1603,QLEARFEPKQRNFRKRELDFEKLFANMPDY,"Imcroporin (Arthropods, animals)",Staphylococcus aureus,['20μg/ml'],Gram-positivo,0,20.00,30.0,6.667,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1494,1604,GLFVGLAKVAAHNNPAIAEHFQA,"Maculatin 1.2 (Frog, amphibians, animals)",Staphylococcus aureus,['50 μg/ml'],Gram-positivo,0,50.00,23.0,26.087,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1495,1605,AFALKKK,peptide 12 (Derived from B1),Pseudomonas aeruginosa,['>64µg/ml'],Gram-negativo,0,64.00,7.0,28.571,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1496,1606,AFALKKK,peptide 13 (Derived from B1),Pseudomonas aeruginosa,['>64µg/ml'],Gram-negativo,0,64.00,7.0,28.571,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0


In [ ]:
#Continue with only the relevant columns


data = data.loc[:, [ 'Sequence','MIC_processado',"Gram","multiresistant","Name","Target_Organism"]]

print(data)

                                               Sequence  MIC_processado  \
0               TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA            0.50   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
2                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.38   
3                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
4     NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...            3.20   
...                                                 ...             ...   
1493                     QLEARFEPKQRNFRKRELDFEKLFANMPDY           20.00   
1494                            GLFVGLAKVAAHNNPAIAEHFQA           50.00   
1495                                            AFALKKK           64.00   
1496                                            AFALKKK           64.00   
1497                                            KKKLAYA           64.00   

               Gram  multiresistant  \
0     Gram-positivo               0   
1     Gram-negativo  

In [ ]:
# Load the ESM-1b model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


In [ ]:
# Function to convert a peptide sequence into embeddings using ESM
def embed_peptide_sequence_esm(sequence):
    data = [("protein1", sequence)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])

    token_representations = results["representations"][33]

    # Removes the representations of the special token [CLS]
    sequence_representations = token_representations[0, 1: len(sequence) + 1].numpy()

    # Returns the average embeddings of all the tokens in the sequence
    mean_embeddings = np.mean(sequence_representations, axis=0)

    return mean_embeddings


In [ ]:
# Auxiliary function for parallel processing
def process_sequence(seq):
    return embed_peptide_sequence_esm(seq)

In [ ]:
# Split the DataFrame into subsets
def split_dataframe(df, chunk_size):
    return [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

In [ ]:
# Function to process each subset and generate the DataFrame with reduced embeddings
def process_chunk(df_chunk):
    sequences = df_chunk['Sequence'].tolist() #Sequence
    with Pool(cpu_count()) as pool:
        embeddings = list(tqdm(pool.imap(process_sequence, sequences), total=len(sequences)))

    embeddings_matrix = np.vstack(embeddings)
    pca = PCA(n_components=50)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)

    reduced_embeddings_df = pd.DataFrame(reduced_embeddings, columns=[f'PC_{i+1}' for i in range(reduced_embeddings.shape[1])])
    return pd.concat([df_chunk.reset_index(drop=True), reduced_embeddings_df], axis=1)


In [ ]:
# Size of the subset
chunk_size = 3000

# Split the DataFrame
df_chunks = split_dataframe(data, chunk_size)
#df_chunks = split_dataframe(dados, chunk_size) #for 0.05%

In [ ]:
# Process each subset and combine the results
df_processed_chunks = [process_chunk(chunk) for chunk in df_chunks]
df_final = pd.concat(df_processed_chunks, axis=0).reset_index(drop=True)

 91%|█████████ | 1357/1498 [22:23<02:05,  1.13it/s]

In [ ]:

from google.colab import files

In [ ]:
df_final.to_csv("/content/este_embeddings_MIC_ESM-1b.csv", index=False)

In [ ]:
files.download('/content/embeddings_MIC_ESM-1b.csv')

In [ ]:
df_final

,Unnamed: 0,Sequence,Function,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,...,PC_41,PC_42,PC_43,PC_44,PC_45,PC_46,PC_47,PC_48,PC_49,PC_50
0,NaN,KKKALFALWLAFLAKKK,antibacterian group ESKAPE,-0.138828,-0.860507,-0.530137,1.321133,-0.459671,-0.041311,0.407025,...,-0.210574,0.151738,-0.053001,0.096609,0.058363,0.161061,0.157936,0.015113,-0.057124,0.129345
1,NaN,WRPGRWWRPGRWWRPGRWWRPGRW,antibacterian group ESKAPE,-0.425542,2.367922,4.887053,-1.954311,0.709735,-0.118830,-0.182685,...,0.617749,0.072955,0.031804,0.236701,0.080823,0.387004,-0.196269,0.131102,0.171641,0.011582
2,NaN,WRGRWWRGRWWRGRW,antibacterian group ESKAPE,-0.698087,0.051814,3.967878,-1.680986,-0.417841,0.358441,0.340468,...,0.596900,0.320002,0.007535,0.236754,0.156275,0.301302,0.123619,0.036784,0.239809,-0.123875
3,NaN,GKYGFYTHVFRLKKWMQKVIDRFGG,antibacterian group ESKAPE,-0.159979,-0.825506,-1.235398,0.113954,0.438347,-0.941559,0.002389,...,0.407786,0.258055,-0.067394,0.049061,-0.085852,0.021115,-0.018899,0.138532,-0.190339,-0.049724
4,NaN,GNGNLLGGLLRPVLGVVKGLTGGLGKK,antibacterian group ESKAPE,-0.614240,4.135028,-0.726304,-0.560266,-0.275694,1.258033,-0.363572,...,0.179352,-0.207444,-0.068961,0.015917,0.159774,0.441414,-0.014362,0.416767,0.151285,-0.051302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2904,NaN,VIFKVFWRCKAAFC,antibacterian group ESKAPE,-0.500496,-2.960468,-0.440940,0.502327,0.174006,0.557715,-1.288929,...,0.123232,-0.130800,0.157171,-0.156585,-0.040823,0.038134,0.152279,-0.040691,0.024894,-0.104778
2905,NaN,AVKRAVKRFKVHFRCKAAFC,antibacterian group ESKAPE,-0.275332,-1.544039,-0.645264,0.111009,0.290090,-0.323162,-0.029677,...,0.045766,0.007506,0.016552,0.017483,0.097331,0.067298,0.103599,0.021538,-0.021151,0.032924
2906,NaN,AVNIPFKVHFKVHFRCKAAFC,antibacterian group ESKAPE,-0.647128,-1.767466,0.303806,0.877676,1.185211,0.463978,-1.134946,...,0.168605,-0.062638,0.085520,0.046987,0.080330,-0.005748,0.122445,-0.063655,0.085298,-0.028428
2907,NaN,GLLSGVLGAGKKIVAGLSGRA,antibacterian group ESKAPE,-0.531606,3.417577,-1.255099,-0.205704,-0.520591,1.120637,0.152368,...,-0.015368,-0.008795,0.419594,0.214909,0.155718,0.143901,0.119236,0.066752,0.018306,0.114852


In [ ]:
df_merged.to_csv("/content/ESMb1_inteiro.csv", index=False)